In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-3sulyfo8
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-3sulyfo8
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=096dc1539712660b4e0a495f4859f384eabbf2df343ea359f1d744b788a7f09e
  Stored in directory: /tmp/pip-ephem-wheel-cache-bg1uivg_/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


%%writefile depthfirst.cu

In [10]:
%%writefile dfs.cu

#include <iostream>
#include <vector>
#include <stack>
#include <cuda_runtime.h>

using namespace std;

__global__ void dfs_kernel(int *adj_list, int *visited, int num_vertices, int source) {
    int thread_id = threadIdx.x + blockIdx.x * blockDim.x;

    if (thread_id == 0) {
        int stack[1024]; // Assuming maximum stack size of 1024
        int top = -1;

        stack[++top] = source;
        visited[source] = 1;

        while (top >= 0) {
            int curr_vertex = stack[top--];
            printf("%d ", curr_vertex);

            for (int i = 0; i < num_vertices; ++i) {
                int neighbour = adj_list[curr_vertex * num_vertices + i];
                if (neighbour && !visited[i]) {
                    visited[i] = 1;
                    stack[++top] = i;
                }
            }
        }
    }
}



int main() {
    int num_vertices, num_edges, source;
    cout << "Enter number of vertices: ";
    cin >> num_vertices;
    cout << "Enter number of edges: ";
    cin >> num_edges;
    cout << "Enter source node: ";
    cin >> source;

    // Input validation
    if (source < 0 || source >= num_vertices) {
        cout << "Invalid source node!" << endl;
        return 1;
    }

    vector<int> adj_list(num_vertices * num_vertices, 0);
    cout << "Enter the edges:" << endl;
    for (int i = 0; i < num_edges; i++) {
        int u, v;
        cin >> u >> v;
        // Input validation for edges
        if (u < 0 || u >= num_vertices || v < 0 || v >= num_vertices) {
            cout << "Invalid edge: " << u << " " << v << endl;
            return 1;
        }
        adj_list[u * num_vertices + v] = 1;
        adj_list[v * num_vertices + u] = 1;
    }

    int *d_adj_list, *d_visited;
    cudaMalloc((void **)&d_adj_list, num_vertices * num_vertices * sizeof(int));
    cudaMalloc((void **)&d_visited, num_vertices * sizeof(int));

    cudaMemcpy(d_adj_list, &adj_list[0], num_vertices * num_vertices * sizeof(int), cudaMemcpyHostToDevice);

    int *visited = new int[num_vertices];
    for (int i = 0; i < num_vertices; i++) {
        visited[i] = 0;
    }

    cudaMemcpy(d_visited, visited, num_vertices * sizeof(int), cudaMemcpyHostToDevice);

    // CUDA event creation for measuring execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Launch DFS kernel
    dfs_kernel<<<1, 1>>>(d_adj_list, d_visited, num_vertices, source);

    // Record and calculate execution time
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << endl << "Execution Time: " << milliseconds << " ms" << endl;

    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_adj_list);
    cudaFree(d_visited);
    delete[] visited;

    return 0;
}


Overwriting dfs.cu


In [11]:
!nvcc dfs.cu -o dfs


In [13]:
!./dfs

Enter number of vertices: 6
Enter number of edges: 6
Enter source node: 0
Enter the edges:
0 1
0 2
1 3
1 4
2 4
2 5
0 2 5 4 1 3 
Execution Time: 127.992 ms
